### Парсинг статей с сайта *lenta.ru*
- заполнить таблицу с метаинформацией о статьях: автор, название, тип, *url*
- заполнить таблицу с леммами в каждой статье и их количеством
- заполнить таблицу с леммами по всем статьям: лемма, длина, число слогов, тип
- используя запросы SQL, посчитать сложность (LIX, FRE) статей различной тематики,найти топ авторов

In [34]:
# Подключаем библиотеки.
import mysql.connector
import requests
import re
import pandas as pd

In [87]:
con = mysql.connector.connect(host="fox", database="new_schema", user="root", password="300955")

In [88]:
cur = con.cursor(dictionary=True)

Получим данные заполнения таблицы из статей сайта lenta.ru за март 2018 года

In [39]:
cur.execute("truncate articles")
cur.execute("truncate vocab")
cur.execute("truncate art_tokens")
#con.commit()
lemms = []
data = open('/home/lisa/proglearning/python/hse/lenta.ru/lenta.ru/lenta_short.tsv')
aid = 1

for line in data:
    # достаём метаданные статьи
    attrs = line.split('\t')
    #for i in range(len(attrs)):
    #    print(i, attrs[i])
    #break
    title = attrs[4].strip("«»")
    author = attrs[1]
    url = attrs[5]
    atype = url.split('/')[3]
    filename = url.strip('/').split('/')[-1]

    print(aid, filename)

    # достаём леммы для для каждой статьи, считаем количество
    ar_token_cnt = {}
    ar_file = open("/home/lisa/proglearning/python/hse/lenta.ru/lenta.ru/parsed.text/2018/03/%s.tsv" % (filename))
    for wline in ar_file:
        wline_s = wline.split('\t')
        if len(wline_s) < 3:
            continue
        lemm = wline_s[1]
        if lemm == '-':
            lemm = wline_s[0].strip()
            if len(lemm) != 1 or not re.match("[,.!?:;'\"-]", lemm):
                continue
        if lemm not in lemms:
            lemms.append(lemm)
        if not lemm in ar_token_cnt:
            ar_token_cnt[lemm]=0
        ar_token_cnt[lemm] += 1

    for lemm in ar_token_cnt.keys():
        cur.execute("INSERT into art_tokens (art_id, token, count) values (%s, %s, %s)",
                   (aid, lemm, ar_token_cnt[lemm]))
        #con.commit()

    req = """INSERT INTO articles (id, title, author, url, type, filename)
        VALUES (%s, %s, %s, %s, %s, %s)"""
    args = (aid, title,author,url, atype, filename)
    #print(args)
    cur.execute(req, args)
    #con.commit()
    aid += 1
    #if aid > 10:
     #   break

# делаем общий словарь лемм: лемма, длина, число слогов, тип
for lemm in lemms:
    #print(lemm)
    #continue
    length = len(lemm)
    syllables = len(re.split("[аеёиоуыэюяaeiouy]", lemm))-1
    ltype = "word"
    if re.match("[,.!?:;'\"-]", lemm):
        ltype = "non-word"
    #print("INSERT", lemm, length, syllables, ltype)
    cur.execute("insert into vocab (token, length, syllables, type) values (%s, %s, %s, %s)",
               (lemm, length, syllables, ltype))
    #con.commit()

con.commit()    

1 crimea
2 haiti_hell
3 slavic
4 free_love
5 sony
6 dovlatov
7 gumilev
8 crimeahousing
9 duhognya
10 mirai
11 schizo
12 sluzhba
13 postavki
14 ball
15 guerra
16 amnistia
17 kachestvo
18 uspehmaska
19 sinestia
20 dahish_ghetto
21 zapahi
22 uyti
23 periods
24 ribka
25 drone
26 ibu
27 lvr_new
28 hudie
29 zvezdi
30 spider
31 spasateli
32 russian_woman
33 harassement
34 blackpanter
35 gasprom_naftogaz
36 ukranian_smartass
37 rachelwood
38 native
39 shark
40 germany_okr
41 trump
42 newbuiltstats
43 prosto_narkokyrier
44 denunciation
45 tarantino
46 sober
47 ali_ali
48 lawyer
49 rasnitsynet
50 hijab
51 eliterealtyreview
52 grudinin
53 leon
54 olympic_new
55 pokoi
56 efimova_smi
57 demo
58 poslanie
59 piteravto
60 povorot
61 enemy
62 ultimate_devotion
63 urban
64 ditatko
65 sea_animal
66 spy0day
67 reestr
68 sobyanin
69 plesh
70 ipo
71 avakovhouse
72 gaz_nafta
73 border
74 krush_tokyo
75 dispute
76 turist
77 sarmat
78 kinzhal
79 armament
80 haute_horlogerie
81 selfi
82 gun
83 alkotest
84 comme

633 china_gas
634 nechegotrogatsobaku
635 topor
636 finale
637 watson_mistake
638 otravlenie
639 thewall
640 wheels
641 proxorov
642 ultimate_robbery
643 krymih
644 sexism
645 yadrovo
646 italy_elections
647 beirut
648 robotcongrats
649 domogar
650 uspela
651 guba_ne_dura
652 putin_football
653 falsealert
654 pristav
655 minute
656 navoz
657 oplot
658 prom
659 oscarisland
660 ploti_nologi_esh_plohoi_eda
661 antiporn
662 otdai_kisu
663 general
664 dolg
665 4glte
666 naming
667 mour_tv
668 mk
669 docha
670 rape
671 iphoneputin
672 harasta
673 igil
674 drevniyles
675 cryptolit
676 moscowbuyers
677 notrth_kore_nukes
678 presents
679 cancer
680 freed
681 mutko_protiv
682 goldenratio
683 ddos
684 podderjka
685 aeroflotbonus
686 usa_syria
687 saratov
688 oper
689 jeep
690 opasnost
691 educationkids
692 get_ready
693 verblud
694 kushat_podano
695 prizzz
696 yellowsock
697 russport
698 embiggens
699 detki
700 an26
701 vesttest
702 vot_i_pokushali
703 an26_prichina
704 pogrozili_palchikom
705 sa

1235 caragher
1236 loza
1237 russia_turkey
1238 sanctions
1239 cryptofake
1240 nagrada
1241 snoopdug
1242 akuloborets
1243 kim_snake
1244 cheaprent
1245 su25
1246 savidi_illegal
1247 starttrack
1248 bombi
1249 molodtsy
1250 duffers
1251 na_ni
1252 dress
1253 plane_airport
1254 shta
1255 punish_a_muslim_day
1256 restaurant_charge
1257 peskov_purga
1258 longlivingmoscovites
1259 budget_salary
1260 stars
1261 savvidi_arest
1262 wunderkind
1263 skripal
1264 busstops
1265 frida
1266 no_punishment
1267 obiski
1268 rost_zarplat
1269 saudi_aramco
1270 bones
1271 cryptocurrency
1272 satan2
1273 zombie
1274 blagodat
1275 miting
1276 khabarovsk
1277 i_vot_opyat
1278 ushel
1279 neeshpodumoi
1280 slutsky_job
1281 miner
1282 annakarenina
1283 shuvalov
1284 realworld
1285 fingerpiercing
1286 tabakov
1287 siroti
1288 olegtabakov
1289 givenchy_death
1290 teslasun
1291 berezovski
1292 ekzorzist
1293 otrava
1294 okgoogle
1295 nisheta
1296 mouse
1297 rybka_za_pivo
1298 sudii
1299 putin_skripal
1300 space


1811 go_away
1812 last_video
1813 motiv
1814 yamaldengo
1815 nacbank
1816 gorerybak
1817 neymar
1818 vasiljev
1819 sevastopol
1820 otvet
1821 sar
1822 vata
1823 crypto
1824 deviz_pmef
1825 dont_criticise_them
1826 mundepi
1827 murmur
1828 neokrylyaet
1829 france_skripal
1830 jimenez
1831 armata
1832 gold_price
1833 alliance
1834 rekord
1835 business
1836 pornoparodies
1837 dolg
1838 knees
1839 thud
1840 pole
1841 fashionbrunch
1842 price
1843 fico
1844 zakon
1845 prigozhin
1846 rus_gold
1847 kepler
1848 ne_zhdem_a_gotovimsya
1849 city
1850 russia_protiv_vseh
1851 badpolice
1852 kuricy
1853 costs
1854 neprostotak
1855 zhku
1856 debaty
1857 kerzh_couch
1858 vodka
1859 text
1860 new_auto
1861 brat
1862 syrian_boys
1863 tolpa
1864 ogloh
1865 attack
1866 usspeh
1867 cat_case
1868 vot_eto_povorot
1869 svoya
1870 purple
1871 lavrov_pal
1872 luchsh
1873 buldozer
1874 crystal
1875 atletico
1876 sumka
1877 kultura
1878 princess
1879 fas
1880 deti
1881 pop
1882 bistro
1883 uygra_kassatsia
1884 pr

2401 mladenci_v_morozilke
2402 xi_jinping
2403 tweets
2404 boeingmh370
2405 gold
2406 fasting
2407 tanker
2408 ibra_god
2409 prostitutki_again
2410 gruntmetro
2411 note
2412 gibdd
2413 hero
2414 morriszhykova
2415 private_funeral
2416 muravied
2417 pereigral
2418 rezidiv
2419 slutsk
2420 klyuchevskie
2421 dauria
2422 novichok
2423 gasukr
2424 pictures
2425 sadnews
2426 sarkozy
2427 messi_rost
2428 weinstein
2429 stretchycondo
2430 ivanzinko
2431 otmena
2432 bezos
2433 opros
2434 boykot
2435 kukla
2436 pu238
2437 oboznalis
2438 secret
2439 trump_congrats
2440 seamonster
2441 affordability
2442 telegram
2443 debilu
2444 weinshtein
2445 ronaldy_china
2446 vuboru
2447 tribunal
2448 telega
2449 kaktebetakoeilonmask
2450 subbotnik
2451 salary
2452 matkapital
2453 turkey_tortures
2454 proton
2455 bentley_fingerprint
2456 stranger
2457 golosovanie
2458 putiniphone
2459 iphone
2460 kondratjev
2461 poland
2462 budget
2463 rybkuzhalko
2464 charlie
2465 ukrklan
2466 babah
2467 khleb
2468 rosatombi

2962 otstavka
2963 vita
2964 ukr_exodus
2965 lucky
2966 gritc
2967 minfin
2968 fakenews
2969 renov
2970 minicondo
2971 disgrace
2972 volgograd
2973 nekalashnikov
2974 msu
2975 weird_goal
2976 ambulance
2977 s_h_t_a
2978 duckduckduck
2979 willismovie
2980 strip_search
2981 isis_soldier
2982 muslim_disney
2983 trade_war
2984 leave
2985 texas_shooter
2986 kokainum
2987 harassment
2988 againstcancer
2989 von
2990 mutant
2991 gazprom
2992 bomba
2993 russia_ten
2994 yje_mojno
2995 sex_voodoo_djset
2996 uhta
2997 irishmodels
2998 elviraaaaaa
2999 nabeg
3000 snake
3001 roshen
3002 aeroflotuspenskaya
3003 miner
3004 mat
3005 syriabusiness
3006 fish
3007 dancingowl
3008 rally
3009 russia_ukraine
3010 izuchat
3011 row
3012 svoboda
3013 guccifer
3014 beauty
3015 arest
3016 kic8462852
3017 ukraine_gas
3018 student
3019 syrian_flashbacks
3020 kimface
3021 brazil
3022 animals
3023 zhenovach
3024 musk
3025 chm
3026 russia_brasil
3027 cherchesov
3028 mustache
3029 zhkh
3030 corbyn
3031 messi
3032 vinto

3544 peskov_tuleev
3545 cifra
3546 racism_again
3547 skripal
3548 naominose
3549 boletnelzya
3550 fire
3551 khl_cuts
3552 newshoppingmalls
3553 chvk
3554 peskov_confrontation
3555 buran
3556 notification
3557 139mln
3558 patriot
3559 bitegate
3560 us_lng
3561 hmfailed
3562 sobolev
3563 ethiopian_football
3564 najiva
3565 pokazania
3566 malchikilidevochka
3567 milky_war
3568 hunger_strike
3569 arestnadezhda
3570 vse_tainoe_stanovitsja_javnym
3571 stuff
3572 medvedev
3573 anofriev
3574 facebook
3575 ntechlab
3576 ak
3577 matza
3578 orangesochi
3579 human_rights
3580 vmoskvu
3581 ophan
3582 karyakin
3583 ne_rasskazhal
3584 putin_checkup
3585 eto_ne_mi
3586 bus
3587 navigatortykuda
3588 za_mat_sorry
3589 svyaz_expo
3590 bankruptcy
3591 terminator
3592 volgograd
3593 teachers
3594 lukoilbuyback
3595 reztsova
3596 vstre4a
3597 edinoross
3598 povar
3599 bears
3600 podojgli
3601 drift
3602 asianlabour
3603 polyus
3604 putin_lies
3605 pory4enie
3606 pressure
3607 dokument
3608 lawyers
3609 sove

Посчитаем число предложений в каждой статье и запишем в articles

In [40]:
cur.execute("SELECT sum(count) as sum, art_id FROM art_tokens WHERE token in ('.','!','?') GROUP BY art_id")
for record in cur.fetchall():
    cur.execute("UPDATE articles SET sent_num = %s WHERE id = %s",(int(record['sum']), record['art_id']))
con.commit()

Посчитаем число слов в каждой статье и запишем в articles

In [41]:
cur.execute("""SELECT sum(count) as sum, art_id
            FROM art_tokens JOIN vocab 
            ON art_tokens.token=vocab.token 
            WHERE type='word' group by art_id""")
for record in cur.fetchall():
    cur.execute("UPDATE articles SET words_num = %s WHERE id = %s",(int(record['sum']), record['art_id']))
con.commit()

Посчитаем коэффициент сложности текста
${\displaystyle {\text{LIX}}={\frac {A}{B}}+{\frac {C\cdot 100}{A}}}$  
A — количество слов в тексте,  
B — количество предложений в тексте,  
C — количество слов длиннее 6 букв.  

In [42]:
# джойним таблички по двум запросам и считаем формулу: 
# 1) С в каждой статье: джойним словарь и инфо по числу слов для статей 
# 2) А и B в каждой статье
cur.execute("""SELECT artinfo.id, A/B+100*C/A as lix 
            FROM 
            (SELECT sum(count) as C, art_id 
            FROM art_tokens 
            JOIN vocab ON art_tokens.token=vocab.token 
            WHERE length>6 
            GROUP BY art_id) as longwords 
            JOIN 
            (SELECT id, sent_num as B, words_num as A 
            FROM articles) as artinfo 
            ON artinfo.id = longwords.art_id""")

for record in cur.fetchall():
    cur.execute("UPDATE articles SET LIX = %s WHERE id = %s",(int(record['lix']), record['id']))
con.commit()

Посчитаем коэффициент сложности FRE = 206,835 − 1,3 × ASL − 60,1 × ASW  
ASL - средняя длина предложения в словах, ASL = A/B  
ASW - средняя длина слова в слогах, ASW = S/A, S - число слогов в тексте  
${\displaystyle {\text{FRE}}={206.835}-{1.3\cdot {\frac {A}{B}}}-{60.1\cdot{\frac {S}{A}}}}$ 

In [43]:
# джойним таблички по двум запросам и считаем формулу: 
# 1) S в каждой статье: джойним словарь и инфо по числу слов для статей 
# 2) А и B в каждой статье
cur.execute("""SELECT artinfo.id, 206.835-1.3*A/B-60.1*S/A as fre 
            FROM 
            (SELECT sum(syllables) as S, art_id 
            FROM art_tokens 
            JOIN vocab ON art_tokens.token=vocab.token 
            GROUP BY art_id) as syll 
            JOIN 
            (SELECT id, sent_num as B, words_num as A 
            FROM articles) as artinfo 
            ON artinfo.id = syll.art_id""")

for record in cur.fetchall():
    cur.execute("UPDATE articles SET FRE = %s WHERE id = %s",(int(record['fre']), record['id']))
con.commit()

Посмотрим, что получили:

In [82]:
cur.execute("SELECT * FROM articles")
res = cur.fetchall()

data = pd.DataFrame(res)

data.head()

,FRE,LIX,author,filename,id,sent_num,title,type,url,words_num
0,59.0,59.0,-,crimea,1,6.0,Черная Таврида,photo,http://lenta.ru/photo/2018/03/01/crimea/,110.0
1,114.0,47.0,Леонид Пашковский,haiti_hell,2,208.0,Это проклятое место,articles,http://lenta.ru/articles/2018/03/01/haiti_hell/,2955.0
2,116.0,48.0,Михаил Карпов,slavic,3,132.0,"Мы, славяне, много страдали",articles,http://lenta.ru/articles/2018/03/01/slavic/,1195.0
3,105.0,61.0,Александр Волобуев,free_love,4,89.0,Вышли из офиса,articles,http://lenta.ru/articles/2018/03/01/free_love/,1402.0
4,54.0,50.0,-,sony,5,5.0,Тьма рядом,photo,http://lenta.ru/photo/2018/03/01/sony/,37.0


Посмотрим на характеристики текстов для статей разных типов

In [67]:
cur.execute("""SELECT type, count(id) as cnt, avg(LIX) as LIX, avg(FRE) as FRE, avg(words_num) as words 
            FROM articles
            GROUP BY type
            ORDER BY cnt DESC""")

res = cur.fetchall()

In [80]:
type_data = pd.DataFrame(res)
type_data.LIX = type_data.LIX.astype(int)
type_data.FRE = type_data.FRE.astype(int)
type_data.words = type_data.words.astype(int)
type_data

,FRE,LIX,cnt,type,words
0,75,58,3686,news,169
1,104,56,226,articles,1387
2,62,56,49,photo,103
3,73,70,16,pressrelease,279
4,94,64,7,brief,703
5,73,59,4,video,271
6,105,50,3,extlink,1094


In [76]:
cur.execute("SELECT * FROM articles WHERE type = 'extlink'")
for r in cur.fetchall():
    print(r['id'],r['url'])

768 http://lenta.ru/extlink/2018/03/07/kandidat/
1065 http://lenta.ru/extlink/2018/03/10/pornopoly/
3674 http://lenta.ru/extlink/2018/03/29/moslenta/


Посмотрим авторов, написавших больше всего статей

In [106]:
cur.execute("""SELECT author, cnt
            FROM 
            (SELECT author, count(id) as cnt
            FROM articles 
            WHERE NOT author = '-' 
            GROUP BY author ) as auth
            ORDER BY cnt DESC""")
res = cur.fetchall()
authors= pd.DataFrame(res)

authors.head(10)

,author,cnt
0,Вероника Гудкова,8
1,Владимир Шарапов,7
2,Дарья Гулина,7
3,Денис Рузаев,7
4,Николай Костюков,6
5,Анастасия Борисова,5
6,Андрей Борисов,5
7,Владимир Веретенников,5
8,Анна Комиссарова,5
9,Егор Поляков,5
